In [ ]:
%%capture
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import seaborn as sns
import nltk 
import heapq
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

data_url="https://raw.githubusercontent.com/liraop/recinfo_lab2/master/data/results.csv"
data = pd.read_csv(data_url).replace(np.nan, '', regex=True)
documents = data.text.count()

def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    year_pattern = rpt(r'\d{4}')
    
    patterns = [word_pattern, year_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token)
        words.extend(tokens)
    return words


def build_index(dataset):
    document_index = 0
    index = {"doc_row": []}
    
    for entry in dataset.text:
        document_index = document_index + 1
        index["doc_row"].append(document_index)
            
        for ngram in parse(entry):
                if ngram in index: #is ngram already on index?
                    if document_index in index[ngram]: # is it in the same document?
                        index[ngram][document_index] = index[ngram][document_index] + 1
                    else: # nope
                        index[ngram][document_index] = 1 
                else: # no, sir
                    index[ngram] = {document_index: 1}
    
    return index
                        
index = build_index(data)

queries = ["juíza","federal","governo","Brasil","presidente"]

def get_inter_ab(index, word1, word2):
    docs = 0.0
    
    if word1 not in index or word2 not in index:
        return 0.0
    
    word1_inverted_list = index[word1].keys()
    word2_inverted_list = index[word2].keys()
    
    for doc_id in word1_inverted_list:
        if doc_id in word2_inverted_list:
            docs += 1.0
    
    return docs

def get_n_word(index,word):
    if word in index:
        inverted_list = index[word]
        return float(len(inverted_list))
    else:
        return 0.0

#### Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que? (20 pts.)

In [ ]:
def dice(index, word1, word2):
    n_a = get_n_word(index,word1)
    n_b = get_n_word(index,word2)
    n_ab = get_inter_ab(index,word1,word2)
    
    dom = n_a + n_b            
    
    if dom == 0.0:
        return 0
    else:
        return n_ab/dom

def emim(index, word1, word2):
    n_a = get_n_word(index,word1)
    n_b = get_n_word(index,word2)
    n_ab = get_inter_ab(index,word1,word2)
    
    
    dom = n_a * n_b
    factor = 0
    
    if (dom != 0):
        factor = N * (n_ab/dom)
        
    if factor == 0: return 0
       
    return n_ab * math.log10(factor)


def x2(index, word1, word2):
    n_a = get_n_word(index,word1)
    n_b = get_n_word(index,word2)
    n_ab = get_inter_ab(index,word1,word2)
    
    dom = n_a * n_b
    
    if (dom == 0): return 0
    
    num = math.pow((n_ab - (1/N)*n_a*n_b),2)
    
    return num/dom
    
def mim(index, word1, word2):
    n_a = get_n_word(index,word1)
    n_b = get_n_word(index,word2)
    n_ab = get_inter_ab(index,word1,word2)
    
    dom = n_a * n_b            
    
    if dom == 0.0:
        return 0
    else:
        return n_ab/dom

#### De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a qualidade dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta. (25 pts.)